# Topic Modeling with LDA

In this notebook, I'll take my processed coca data (punctuation removed; stopwords removed; no links, hashtags, or emojis; and lemmatized) and use pyLDAvis to start exploring some of the underlying topics in this corpus. 

pyLDAvis is a great tool to both get top keywords for each topic as well as *visualize* these topics in relation to one another. 

## pyLDAvis Topic Modeling

In [69]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#sklearn
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import warnings
warnings.filterwarnings('ignore')

import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

import getout_of_text_3 as got3

from tqdm.notebook import tqdm
from tqdm import tqdm

## Read the COCA text using `got3`

In [70]:
ls -la ../../data/english-corpora.org/coca/

total 16
drwx------@ 11 ejacquot  staff   352 Nov 10 13:02 ./
drwxr-xr-x   7 ejacquot  staff   224 Nov 10 13:08 ../
-rw-r--r--@  1 ejacquot  staff  6148 Nov 10 13:02 .DS_Store
drwx------@ 32 ejacquot  staff  1024 Sep 12 14:18 text_acad_isi/
drwx------@ 36 ejacquot  staff  1152 Sep 12 14:18 text_blog_wih/
drwx------@ 32 ejacquot  staff  1024 Sep 12 14:18 text_fic_jjw/
drwx------@ 32 ejacquot  staff  1024 Sep 12 14:18 text_mag_jgr/
drwx------@ 32 ejacquot  staff  1024 Sep 12 14:19 text_news_nne/
drwx------@ 32 ejacquot  staff  1024 Sep 12 14:19 text_spok_yuv/
drwx------@ 32 ejacquot  staff  1024 Sep 12 14:19 text_tvm_nwh/
drwx------@ 36 ejacquot  staff  1152 Sep 12 14:19 text_web_ywv/


In [62]:
coca_corpus = got3.read_corpus('../../data/english-corpora.org/coca/')


COCA Genres:   0%|          | 0/8 [00:00<?, ?genre/s]

Processing genre: mag


COCA Genres:  12%|█▎        | 1/8 [00:01<00:07,  1.08s/genre]

Finished genre: mag (total files: 30)
Processing genre: web


COCA Genres:  25%|██▌       | 2/8 [00:02<00:06,  1.07s/genre]

...

Finished genre: blog (total files: 34)
Processing genre: fic


COCA Genres:  88%|████████▊ | 7/8 [00:07<00:01,  1.01s/genre]

Finished genre: fic (total files: 30)
Processing genre: tvm


COCA Genres: 100%|██████████| 8/8 [00:08<00:00,  1.04s/genre]

Finished genre: tvm (total files: 30)


### Search for a keyword to find topic distributions

The classic example of `bank` whether it's a financial bank or a river bank. We get kwic results from COCA for `bank` and then use LDA

In [71]:
keyword='best system'
before = pd.Timestamp.now()
bovine_kwic = got3.search_keyword_corpus(keyword, coca_corpus, 
                                            case_sensitive=False,
                                            show_context=True, 
                                            context_words=15,
                                            output='json',
                                            parallel=True)
after = pd.Timestamp.now()
print('time elapsed:', after - before)

time elapsed: 0 days 00:00:22.006559


In [72]:
# Remove keys from kwic_results where the value is an empty dict
bovine_kwic = {k: v for k, v in bovine_kwic.items() if v}

In [73]:
bovine_kwic

{'mag_1993': {'902_1': 'Automobile Manufacturers Association ( AAMA ) . For consumers , it promises to provide the **best system** to date for choosing oil most appropriate for car engines , particularly newer models .',
  '1278_1': "new republic 's existence , a profound religious purpose permeated American education ( The One **Best System** , 1974 ) . Education without Christian prayer , the exposition and memorization of the",
  '1278_2': 'Terry M. Moe , who do not agree that is resulted in the " one **best system** " ( Politics , Markets and America \'s Schools , 1990 ) . In the'},
 'mag_1992': {'902_1': ': <p> Number one , take it to pieces . Number two , find the **best system** in the world that works , study it , take it to pieces . Number'},
 'mag_1991': {'1334_1': "the Troy-Bilt of the two self-propelled models I tested . The Troy-Bilt also has the **best system** I 've encountered for raising and lowering the height of the mower . Instead of"},
 'mag_1995': {'1611_1': 'GOP 

### Lemmatize and clean the kwic results

In [74]:
import re
import string
from typing import Iterable, List, Dict, Any, Optional

# If you already have a stopword set defined elsewhere you can pass it in; otherwise we build one.
def build_stopwords(extra: Optional[Iterable[str]] = None):
    try:
        from nltk.corpus import stopwords
        try:
            _ = stopwords.words('english')
        except LookupError:
            import nltk
            nltk.download('stopwords')
        base = set(stopwords.words('english'))
    except Exception:
        # Fallback minimal list if nltk not available
        base = {
            'the','a','an','and','or','to','of','in','on','for','at','by','with','from','that',
            'this','is','was','are','were','be','been','it','as','but','if','than','then','so',
            'such','into','its','their','there','here','over','under','out','up','down','not'
        }
    if extra:
        base.update(w.lower() for w in extra)
    return base

def load_spacy(model_name: str = "en_core_web_sm"):
    """Load spaCy model (download if missing)."""
    import importlib
    try:
        import spacy
        return spacy.load(model_name)
    except OSError:
        try:
            from spacy.cli import download
            download(model_name)
            import spacy
            from tqdm import tqdm
            return spacy.load(model_name)
        except Exception as e:
            raise RuntimeError(f"Could not load or download spaCy model '{model_name}': {e}")

# Precompile regexes
URL_RE = re.compile(r'https?://\\S+|www\\.\\S+', re.IGNORECASE)
HASHTAG_RE = re.compile(r'#[A-Za-z0-9_]+')
HANDLE_RE = re.compile(r'@[A-Za-z0-9_]+')
# Basic emoji / symbol range (broad)
EMOJI_RE = re.compile(r'[\\U00010000-\\U0010ffff]', flags=re.UNICODE)
# Markdown bold markers **word**
BOLD_MARK_RE = re.compile(r'\\*\\*')

# Punctuation translator (remove all punctuation chars)
PUNCT_TABLE = str.maketrans('', '', string.punctuation)

def clean_context(text: str,
                  nlp=None,
                  stopwords_set: Optional[set] = None,
                  keep_keyword: Optional[str] = None,
                  min_len: int = 2) -> str:
    """
    Clean a single KWIC context string.
    
    Steps:
      - Lowercase
      - Remove markdown bold markers (**)
      - Remove URLs, hashtags, handles
      - Remove emojis (basic range)
      - Strip any remaining asterisks
      - Remove punctuation
      - Tokenize (simple split after punctuation removal)
      - Drop stopwords & short tokens
      - Lemmatize (spaCy) if nlp supplied
      - Optionally keep the keyword even if filtered
    
    Returns a space-separated string of lemmas/tokens.
    """
    if not isinstance(text, str):
        text = str(text)
    if stopwords_set is None:
        stopwords_set = build_stopwords()
    lowered = text.lower()
    lowered = BOLD_MARK_RE.sub('', lowered)  # remove ** markers
    lowered = URL_RE.sub(' ', lowered)
    lowered = HASHTAG_RE.sub(' ', lowered)
    lowered = HANDLE_RE.sub(' ', lowered)
    lowered = EMOJI_RE.sub(' ', lowered)
    # Remove any leftover asterisks (in case of unmatched markdown)
    lowered = lowered.replace('*', ' ')
    # Replace newlines with space
    lowered = lowered.replace('\\n', ' ')
    # Remove punctuation
    no_punct = lowered.translate(PUNCT_TABLE)
    # Collapse multiple spaces
    no_punct = re.sub(r'\\s+', ' ', no_punct).strip()
    if not no_punct:
        return ''
    tokens = no_punct.split()
    if nlp is not None:
        # Lemmatize
        doc = nlp(' '.join(tokens))
        lemmas = []
        for token in doc:
            lemma = token.lemma_.lower()
            # Remove pronoun artifacts
            if lemma == '-pron-':
                continue
            lemmas.append(lemma)
        tokens = lemmas
    cleaned = []
    keep_kw = keep_keyword.lower() if keep_keyword else None
    for tok in tokens:
        if keep_kw and tok == keep_kw:
            cleaned.append(tok)
            continue
        if tok in stopwords_set:
            continue
        if len(tok) < min_len:
            continue
        if tok.isnumeric():
            continue
        cleaned.append(tok)
    return ' '.join(cleaned)

def kwic_dict_to_docs_seq(kwic_dict: Dict[str, Any],
                      nlp=None,
                      stopwords_set: Optional[set] = None,
                      keep_keyword: Optional[str] = None,
                      min_len: int = 2,
                      show_progress: bool = False) -> List[str]:
    """
    Flatten the kwic result (genre_year -> list[dict{'context': ...}]) into a list of cleaned strings.
    Skips empty cleaned results.
    If show_progress is True, uses tqdm to show a progress bar.
    """
    docs = []
    items = list(kwic_dict.items())
    iterator = items
    if show_progress:
        iterator = tqdm(items, desc="Cleaning KWIC contexts")
    for genre_year, entries in tqdm(iterator, desc="Cleaning KWIC contexts", total=len(items)):
        if isinstance(entries, list):
            for item in entries:
                ctx = item.get('full_text') if isinstance(item, dict) else str(item)
                cleaned = clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                                        keep_keyword=keep_keyword, min_len=min_len)
                if cleaned:
                    docs.append(cleaned)
        else:
            # Unexpected structure fallback
            ctx = str(entries)
            cleaned = clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                                    keep_keyword=keep_keyword, min_len=min_len)
            if cleaned:
                docs.append(cleaned)
    return docs


In [101]:
import multiprocessing
from functools import partial
from concurrent.futures import ProcessPoolExecutor
import os

from typing import Dict, Any, Optional, List

def kwic_dict_to_docs(kwic_dict: Dict[str, Any],
                      nlp=None,
                      stopwords_set: Optional[set] = None,
                      keep_keyword: Optional[str] = None,
                      min_len: int = 2,
                      show_progress: bool = False,
                      parallel: bool = False,
                      n_jobs: Optional[int] = None,
                      chunk_size: int = 100) -> List[str]:
    """
    Flatten the kwic result (genre_year -> list[dict{'context': ...}]) into a list of cleaned strings.
    Skips empty cleaned results.
    
    Args:
        kwic_dict: The KWIC results dictionary
        nlp: spaCy model for lemmatization
        stopwords_set: Set of stopwords to filter
        keep_keyword: Keyword to always keep even if filtered
        min_len: Minimum token length
        show_progress: Show progress bar
        parallel: Use multiprocessing for faster processing
        n_jobs: Number of worker processes (None = auto-detect CPU count)
        chunk_size: Size of chunks for multiprocessing
    
    Returns:
        List of cleaned document strings
    """
    # Collect all entries first
    # Handle both structures: {genre_year: [list]} and {genre_year: {file_id: text}}
    all_entries = []
    for genre_year, entries in kwic_dict.items():
        if isinstance(entries, list):
            # List of dicts with 'context' or 'full_text'
            all_entries.extend(entries)
        elif isinstance(entries, dict):
            # Dict of {file_id: text_string} - extract the text strings
            for file_id, text_content in entries.items():
                if isinstance(text_content, str):
                    all_entries.append({'context': text_content})
                elif isinstance(text_content, dict):
                    all_entries.append(text_content)
                else:
                    all_entries.append({'context': str(text_content)})
        else:
            # Unexpected structure fallback
            all_entries.append({'context': str(entries)})
    
    if not all_entries:
        return []
    
    # Define worker function locally to avoid pickling issues
    def process_entry(entry):
        if isinstance(entry, dict):
            ctx = entry.get('context', entry.get('full_text', ''))
        else:
            ctx = str(entry)
        return clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                            keep_keyword=keep_keyword, min_len=min_len)
    
    if parallel and len(all_entries) > chunk_size:
        try:
            # Use ThreadPoolExecutor instead of ProcessPoolExecutor for Jupyter compatibility
            from concurrent.futures import ThreadPoolExecutor
            
            if n_jobs is None:
                n_jobs = min(4, os.cpu_count() or 1)  # Limit threads for I/O bound tasks
            
            # Process in parallel using threads
            with ThreadPoolExecutor(max_workers=n_jobs) as executor:
                if show_progress:
                    # Submit all tasks and track progress
                    futures = [executor.submit(process_entry, entry) for entry in all_entries]
                    results = []
                    for future in tqdm(futures, desc="Cleaning KWIC contexts (parallel)"):
                        result = future.result()
                        if result:
                            results.append(result)
                else:
                    # Process without progress tracking
                    futures = [executor.submit(process_entry, entry) for entry in all_entries]
                    results = [future.result() for future in futures if future.result()]
            
            return results
            
        except Exception as e:
            print(f"Parallel processing failed ({e}), falling back to sequential processing...")
            # Fall through to sequential processing
    
    # Sequential processing (original implementation)
    docs = []
    iterator = all_entries
    if show_progress:
        iterator = tqdm(all_entries, desc="Cleaning KWIC contexts")
    
    for entry in iterator:
        if isinstance(entry, dict):
            ctx = entry.get('context', entry.get('full_text', ''))
        else:
            ctx = str(entry)
        
        cleaned = clean_context(ctx, nlp=nlp, stopwords_set=stopwords_set,
                               keep_keyword=keep_keyword, min_len=min_len)
        if cleaned:
            docs.append(cleaned)
    
    return docs

In [102]:
import random

random_keys = random.sample(list(bovine_kwic.keys()), 100)
filtered_kwic = {k: bovine_kwic[k] for k in random_keys}
len(filtered_kwic)
# if you want a smaller 

100

In [103]:
filtered_kwic

{'mag_1991': {'1334_1': "the Troy-Bilt of the two self-propelled models I tested . The Troy-Bilt also has the **best system** I 've encountered for raising and lowering the height of the mower . Instead of"},
 'news_1993': {'2109_1': 'president at the University of Texas M.D. Anderson Hospital : " " We have the **best system** in the world , but it is the most costly . The status quo needs'},
 'acad_2013': {'10_1': ', and study of effectiveness . # Selection of the System # To select the **best system** , a committee made up of faculty from different ranks , advisors , administrators ,'},
 'spok_2011': {'536_1': "true . Indeed , the free-market system , up to a point , is the **best system** . It 's what produces the wealth and produces the jobs , the income and",
  '647_1': 'a kind of top to bottom failure of the Democratic Party under Barack Obama. @!MATTHEWS: **Best system** ever . @!Mr-GERSON: Mm-hmm . @!MATTHEWS: Yeah . Gloria : @!Ms-BORGER: Some time @ @'},
 'acad_2003': {'333_1'

In [104]:
# Load spaCy model once
nlp = load_spacy()

# Optionally build stopwords (add domain-specific removals here)
stops = build_stopwords(extra=['court','section','best','system'])  # Add keyword terms to stops

# Use multiprocessing for faster processing on KWIC contexts
proc_kwic = kwic_dict_to_docs(filtered_kwic, nlp=nlp, stopwords_set=stops, 
                             keep_keyword=None,  # Don't force-keep the keyword since we're analyzing around it
                             parallel=True, 
                             show_progress=True,
                             chunk_size=500,
                             n_jobs=9)

print(f"📊 Processed {len(proc_kwic)} KWIC lines")
if proc_kwic:
    print(f"📝 Sample processed: {proc_kwic[0]}")

Cleaning KWIC contexts: 100%|██████████| 144/144 [00:01<00:00, 123.89it/s]

📊 Processed 144 KWIC lines
📝 Sample processed: troybilt two sel propel model test troybilt also good encounter raise lower height mower instead


In [105]:
proc_kwic

['troybilt two sel propel model test troybilt also good encounter raise lower height mower instead',
 'president university texas md anderson hospital good world costly status quo need',
 'study ectiveness selection select good committee make aculty rom di erent rank advisor administrator',
 'true indeed reemarket point good produce wealth produce job income',
 'kind top bottom ailure democratic party barack obama matthews good ever mrgerson mmhmm matthew yeah gloria msborger time',
 'maintain need special recognition help encourage marriage good raise healthy child pass positive value live together',
 'ewer monitor unctionality good operation reliability coretech concept use minimum number sophisticated component',
 'ootage rom tuesday night uh well good mean tell owner need upgrade',
 'many along doctor believe get good medical care world emphasize personal care reedom choice',
 'egypt increase due burgeon use playback technology good bird sound whose digital chip hold highquality re

In [113]:
# Flatten filtered_kwic to get the KWIC concordance lines (context around keyword)
# Structure: {genre_year: {file_id: text_string}}
contexts = []
for genre_year, file_dict in filtered_kwic.items():
    if isinstance(file_dict, dict):
        # Extract text strings from {file_id: text_string} dict
        for file_id, text in file_dict.items():
            if isinstance(text, str):
                contexts.append(text)
            else:
                contexts.append(str(text))
    else:
        # Fallback for unexpected structure
        contexts.append(str(file_dict))

print(f"📊 Extracted {len(contexts)} KWIC concordance lines")
if contexts:
    print(f"📝 Sample context: {contexts[0][:200]}...")


📊 Extracted 144 KWIC concordance lines
📝 Sample context: the Troy-Bilt of the two self-propelled models I tested . The Troy-Bilt also has the **best system** I 've encountered for raising and lowering the height of the mower . Instead of...


In [118]:
contexts

["the Troy-Bilt of the two self-propelled models I tested . The Troy-Bilt also has the **best system** I 've encountered for raising and lowering the height of the mower . Instead of",
 'president at the University of Texas M.D. Anderson Hospital : " " We have the **best system** in the world , but it is the most costly . The status quo needs',
 ', and study of effectiveness . # Selection of the System # To select the **best system** , a committee made up of faculty from different ranks , advisors , administrators ,',
 "true . Indeed , the free-market system , up to a point , is the **best system** . It 's what produces the wealth and produces the jobs , the income and",
 'a kind of top to bottom failure of the Democratic Party under Barack Obama. @!MATTHEWS: **Best system** ever . @!Mr-GERSON: Mm-hmm . @!MATTHEWS: Yeah . Gloria : @!Ms-BORGER: Some time @ @',
 'to maintain and need special recognition to help encourage them . <p> Marriage is the **best system** for raising healthy chil

In [120]:
# Create DataFrame - align contexts with proc_kwic
# Structure: {genre_year: {file_id: text_string}}

print(f"📊 Lengths: contexts={len(contexts)}, proc_kwic={len(proc_kwic)}")

# Re-extract and align in one pass to match filtered results
aligned_contexts = []
aligned_processed = []

for genre_year, file_dict in filtered_kwic.items():
    if isinstance(file_dict, dict):
        # Extract text strings from {file_id: text_string} dict
        for file_id, text in file_dict.items():
            if isinstance(text, str):
                raw_ctx = text
            else:
                raw_ctx = str(text)
            
            # Process this specific context
            proc_ctx = clean_context(raw_ctx, nlp=nlp, stopwords_set=stops, 
                                     keep_keyword=None, min_len=2)
            
            # Only include if we got processed text
            if proc_ctx and proc_ctx.strip():
                aligned_contexts.append(raw_ctx)
                aligned_processed.append(proc_ctx)

print(f"✅ Aligned: {len(aligned_contexts)} concordance lines\n")

df_kwic = pd.DataFrame({
    'clean': aligned_contexts,
    'processed': aligned_processed
})
df_kwic.index.name = 'index'
df_kwic.head()

📊 Lengths: contexts=144, proc_kwic=144
✅ Aligned: 144 concordance lines

✅ Aligned: 144 concordance lines



,clean,processed
index,,
0,the Troy-Bilt of the two self-propelled models...,troybilt two sel propel model test troybilt al...
1,president at the University of Texas M.D. Ande...,president university texas md anderson hospita...
2,", and study of effectiveness . # Selection of ...",study ectiveness selection select good committ...
3,"true . Indeed , the free-market system , up to...",true indeed reemarket point good produce wealt...
4,a kind of top to bottom failure of the Democra...,kind top bottom ailure democratic party barack...


In [121]:
vectorizer = CountVectorizer(token_pattern="\\b[a-z][a-z]+\\b",
                             binary=True,
                             stop_words='english')

In [122]:
dtm_tf = vectorizer.fit_transform(df_kwic.processed)
print(dtm_tf.shape)

(144, 876)


### 2 Topics

- west bank in palestine (river bank), etc versus financial bank, money, loan, etc

In [123]:
%%time

lda_2 = LatentDirichletAllocation(n_components=2, random_state=42)
lda_2.fit(dtm_tf)

CPU times: user 97.2 ms, sys: 2.56 ms, total: 99.8 ms
Wall time: 98.2 ms


,n_components,2
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [124]:
pyLDAvis.lda_model.prepare(lda_2, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.063558  0.0       1        1  55.141619
0     -0.063558  0.0       2        1  44.858381, topic_info=         Term      Freq      Total Category  logprob  loglift
513       new  4.000000   4.000000  Default  30.0000  30.0000
510      need  4.000000   4.000000  Default  29.0000  29.0000
704    school  4.000000   4.000000  Default  28.0000  28.0000
38   american  3.000000   3.000000  Default  27.0000  27.0000
50         ar  5.000000   5.000000  Default  26.0000  26.0000
..        ...       ...        ...      ...      ...      ...
657       ree  2.720620   5.018758   Topic2  -5.5080   0.1893
684       rom  3.382903   7.569006   Topic2  -5.2902  -0.0037
867     world  4.819124  15.899243   Topic2  -4.9363  -0.3920
146      come  2.694837   6.314780   Topic2  -5.5176  -0.0499
176   country  2.176246   4.403375   Topic2  -5.7313   0.0969

[126 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         1  0.628024      able
2         2  0.314012      able
9         1  0.777902       ace
27        1  1.039624       ail
38        2  0.813120  american
...     ...       ...       ...
864       1  0.840277      work
864       2  0.120040      work
867       1  0.691857     world
867       2  0.314480     world
873       1  0.932278      year

[122 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

### 4 Topics

In [125]:
%%time

lda_4 = LatentDirichletAllocation(n_components=4, random_state=42)
lda_4.fit(dtm_tf)

CPU times: user 65.5 ms, sys: 4.28 ms, total: 69.8 ms
Wall time: 68.8 ms


,n_components,4
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [126]:
pyLDAvis.lda_model.prepare(lda_4, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.093920 -0.037003       1        1  36.950376
0      0.085980 -0.065526       2        1  30.376640
2     -0.018169  0.011708       3        1  17.552320
3      0.026109  0.090821       4        1  15.120663, topic_info=        Term       Freq      Total Category  logprob  loglift
867    world  17.000000  17.000000  Default  30.0000  30.0000
568   people   6.000000   6.000000  Default  29.0000  29.0000
391     irst   3.000000   3.000000  Default  28.0000  28.0000
552    ought   2.000000   2.000000  Default  27.0000  27.0000
236      ect   5.000000   5.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
506  natural   0.660400   1.835912   Topic4  -5.8364   0.8667
840  utility   0.660389   1.807800   Topic4  -5.8364   0.8821
395      job   0.660349   3.760194   Topic4  -5.8364   0.1497
873     year   0.660319   3.200943   Topic4  -5.8365   0.3106
121   chance   0.660318   1.807817   Topic4  -5.8365   0.8820

[253 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1         2  0.780532   aama
2         1  0.593332   able
2         2  0.296666   able
9         1  0.411892    ace
9         3  0.411892    ace
...     ...       ...    ...
867       1  0.643131  world
867       2  0.350799  world
873       1  0.624816   year
873       2  0.312408   year
873       4  0.312408   year

[334 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 4])

### 6 Topics

In [127]:
%%time

lda_6 = LatentDirichletAllocation(n_components=6, random_state=42)
lda_6.fit(dtm_tf)

CPU times: user 60.6 ms, sys: 1.79 ms, total: 62.4 ms
Wall time: 61.7 ms


,n_components,6
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [128]:
pyLDAvis.lda_model.prepare(lda_6, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.010685  0.123805       1        1  25.384137
5      0.124367 -0.053303       2        1  24.056189
0     -0.099070 -0.062202       3        1  20.001103
4     -0.031931 -0.033335       4        1  14.378923
2      0.019441  0.036097       5        1  10.009115
3     -0.002122 -0.011061       6        1   6.170533, topic_info=           Term       Freq      Total Category  logprob  loglift
236         ect   5.000000   5.000000  Default  30.0000  30.0000
789       think   6.000000   6.000000  Default  29.0000  29.0000
552       ought   2.000000   2.000000  Default  28.0000  28.0000
867       world  16.000000  16.000000  Default  27.0000  27.0000
513         new   4.000000   4.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
34        allow   0.482989   1.681222   Topic6  -5.2529   1.5381
781  technology   0.482987   2.421355   Topic6  -5.2529   1.1733
682       right   0.482971   2.768676   Topic6  -5.2530   1.0392
354       icial   0.482953   1.540200   Topic6  -5.2530   1.6256
615     produce   0.482931   2.409789   Topic6  -5.2530   1.1780

[353 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1         3  0.789057    aama
2         1  0.312506    able
2         2  0.312506    able
2         4  0.312506    able
6         4  0.833260  accord
...     ...       ...     ...
867       3  0.180211   world
867       4  0.240281   world
872       5  0.887879    yeah
873       1  0.592211    year
873       3  0.296106    year

[419 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 6, 1, 5, 3, 4])

### 8 Topics

In [129]:
%%time

lda_8 = LatentDirichletAllocation(n_components=8, random_state=42)
lda_8.fit(dtm_tf)

CPU times: user 56.1 ms, sys: 2.34 ms, total: 58.4 ms
Wall time: 57.4 ms


,n_components,8
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [130]:
pyLDAvis.lda_model.prepare(lda_8, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.096676 -0.088920       1        1  19.733721
5     -0.064100  0.136895       2        1  17.174186
1     -0.047983 -0.029753       3        1  15.359991
0      0.116197 -0.029398       4        1  14.698278
4     -0.012045 -0.014816       5        1  11.614173
6      0.058249  0.032881       6        1   9.721879
2      0.015041 -0.017746       7        1   6.365218
3      0.031318  0.010858       8        1   5.332555, topic_info=        Term       Freq      Total Category  logprob  loglift
391     irst   3.000000   3.000000  Default  30.0000  30.0000
782     tell   3.000000   3.000000  Default  29.0000  29.0000
236      ect   5.000000   5.000000  Default  28.0000  28.0000
867    world  16.000000  16.000000  Default  27.0000  27.0000
702      say   9.000000   9.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
695       sa   0.502792   3.095271   Topic8  -5.0668   1.1139
553     ound   0.502791   3.095271   Topic8  -5.0668   1.1139
840  utility   0.502791   1.764687   Topic8  -5.0668   1.6758
278      ety   0.502788   1.719488   Topic8  -5.0668   1.7017
34     allow   0.502787   1.719256   Topic8  -5.0668   1.7019

[486 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1         4  0.785806    aama
2         2  0.306224    able
2         4  0.306224    able
2         5  0.306224    able
6         5  0.816671  accord
...     ...       ...     ...
872       1  0.758851    yeah
873       1  0.314400    year
873       3  0.314400    year
873       5  0.314400    year
873       6  0.314400    year

[655 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 6, 2, 1, 5, 7, 3, 4])

### 12 Topics

In [131]:
%%time

lda_12 = LatentDirichletAllocation(n_components=12, random_state=42)
lda_12.fit(dtm_tf)

CPU times: user 55.6 ms, sys: 1.64 ms, total: 57.3 ms
Wall time: 56.7 ms


,n_components,12
,doc_topic_prior,None
,topic_word_prior,None
,learning_method,'batch'
,learning_decay,0.7
,learning_offset,10.0
,max_iter,10
,batch_size,128
,evaluate_every,-1
,total_samples,1000000.0
,perp_tol,0.1


In [132]:
pyLDAvis.lda_model.prepare(lda_12, dtm_tf, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.146416 -0.107746       1        1  14.832329
5     -0.047538 -0.025798       2        1  12.549230
7      0.125931 -0.056118       3        1  11.379190
1      0.056952 -0.068404       4        1  10.885489
0     -0.005598  0.113293       5        1  10.145921
8      0.038434  0.013291       6        1   9.700117
9      0.054462 -0.060557       7        1   7.999332
6     -0.022274  0.052440       8        1   7.258348
10    -0.032261  0.036009       9        1   5.181214
4     -0.004195  0.034328      10        1   3.898221
3     -0.000845  0.063850      11        1   3.886859
2     -0.016652  0.005412      12        1   2.283748, topic_info=       Term      Freq      Total Category  logprob  loglift
236     ect  5.000000   5.000000  Default    30.00  30.0000
448    make  5.000000   5.000000  Default    29.00  29.0000
702     say  9.000000   9.000000  Default    28.00  28.0000
864    work  8.000000   8.000000  Default    27.00  27.0000
568  people  5.000000   5.000000  Default    26.00  26.0000
..      ...       ...        ...      ...      ...      ...
789   think  0.028758   7.279987  Topic12    -7.08  -1.7546
867   world  0.028758  16.876113  Topic12    -7.08  -2.5954
782    tell  0.028758   4.582816  Topic12    -7.08  -1.2918
864    work  0.028758   8.672002  Topic12    -7.08  -1.9296
704  school  0.028758   4.477343  Topic12    -7.08  -1.2685

[646 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1         5  0.786299    aama
2         1  0.602839    able
2         9  0.301420    able
4        10  0.940242     aca
5         8  0.830950  access
...     ...       ...     ...
869       3  0.778315   wrong
872       1  0.753623    yeah
873       3  0.306521    year
873       4  0.306521    year
873       7  0.306521    year

[806 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 6, 8, 2, 1, 9, 10, 7, 11, 5, 4, 3])